In [1]:
# all codes come are changed based on LDR code.
# 用添加噪声的数据进行训练
import argparse
import torch.optim as optim
import os
import yaml
from mcrgan.default import _C as config
from mcrgan.default import update_config
import torch
from mcrgan.trainer import MCRTrainer
from mcrgan.datasets import get_dataloader
from mcrgan.models import get_models
from mcrgan.models import get_BNN_models
from mcrgan.models import get_noise
from mcrgan.models import get_noise_uniform
from mcrgan.models import DiscriminatorMNIST
from mcrgan.models import weights_init_mnist_model_noise

import numpy as np
import torch
import argparse
from mcrgan.datasets import get_dataloader
from mcrgan.models import get_models
from mcrgan.default import _C as config
from mcrgan.default import update_config
from utils.utils import sort_dataset, compute_accuracy, extract_features
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from tqdm import tqdm
import os
from mcrgan.loss import MCRGANloss
import random
random.seed(0)

In [2]:
# 导入Fmnist数据集，进行对比
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
DATASET = 'FMNIST'
ROOT    = 'pth/to/the/dataset'
BATCH_SIZE = 2048
WORKERS = 8
DATA_SAMPLE = 50000
LABEL = 0

dataloader, dataset = get_dataloader(
        data_name=DATASET,
        root=ROOT,
        batch_size=BATCH_SIZE,
        num_workers=WORKERS)

In [3]:
# direct use real_label,normalize var as 0.5
fake_noise_normal = torch.from_numpy(np.random.normal(loc=0.5, scale=0.5, size=[2048,1,32,32])) # var is 1/4
fake_noise_normal = fake_noise_normal.type(torch.FloatTensor)
fake_noise_normal = fake_noise_normal.to(device)
fake_label = torch.from_numpy(np.random.randint(10,size=[2048]))

fake_noise_uniform = np.sqrt(3)*torch.from_numpy(np.random.uniform(0,1,[2048,1,32,32])) # 1/12
fake_noise_uniform = fake_noise_uniform.type(torch.FloatTensor)
fake_noise_uniform = fake_noise_uniform.to(device)

fake_noise_cauchy = torch.from_numpy(np.random.standard_cauchy(size=[2048,1,32,32])+0.5)
fake_noise_cauchy = fake_noise_cauchy.type(torch.FloatTensor)
fake_noise_cauchy = fake_noise_cauchy.to(device)

fake_noise_laplace = torch.from_numpy(np.random.laplace(loc=0.5, scale=1.0, size=[2048,1,32,32]))/ np.sqrt(8) # var is 2
fake_noise_laplace = fake_noise_laplace.type(torch.FloatTensor)
fake_noise_laplace = fake_noise_laplace.to(device)

fake_noise_gumbel = torch.from_numpy(np.random.gumbel(loc=0.5,scale=1.0,size=[2048,1,32,32]))/ np.sqrt(6.58) #pi^2/6 1.645
fake_noise_gumbel = fake_noise_gumbel.type(torch.FloatTensor)
fake_noise_gumbel = fake_noise_gumbel.to(device)
# from 0.01 - 0.99,0 -1
epsilon_noise = 0.1

In [4]:
import collections

def get_netG(netD,netV,net_noise,radius=1):
    models = [netD, netV,net_noise]
    worker_state_dict = [x.state_dict() for x in models]
    weight_keys = list(worker_state_dict[0].keys())
    fed_state_dict = collections.OrderedDict()
    for key in weight_keys:
        fed_state_dict[key] = worker_state_dict[0][key] + radius*worker_state_dict[2][key]*torch.log(1+torch.exp(worker_state_dict[1][key]))
    return fed_state_dict

# new netG already optimized by adam and time learning rate, so we only using netG to update netV
def update_netV(netV,netG,net_noise,fed_state_dict,radius=1):
    models =[netV,netG,net_noise]
    worker_state_dict = [x.state_dict() for x in models]
    weight_keys = list(worker_state_dict[0].keys())
    update_dict = collections.OrderedDict()
    for key in weight_keys:
        update_dict[key] = worker_state_dict[0][key] - (worker_state_dict[1][key]-fed_state_dict[key])*(worker_state_dict[2][key]*radius)/torch.log(1+torch.exp(-worker_state_dict[0][key]))
    return update_dict

# generate different distribution via gaussian init and uniform
def get_cauchy_noise(netD,net_noise1,net_noise2):
    models =[netD,net_noise1, net_noise2]
    worker_state_dict = [x.state_dict() for x in models]
    weight_keys = list(worker_state_dict[0].keys())
    update_dict = collections.OrderedDict()
    for key in weight_keys:
        update_dict[key] = worker_state_dict[1][key] / worker_state_dict[2][key]
    return update_dict
        

In [5]:
#models
# train hyperparameters, batch norm not used var
#store_Var = []
import time
from torch_mimicry.training import scheduler, logger, metric_log

from test_acc import nearsub
import numpy as np
import torch
import argparse
from mcrgan.datasets import get_dataloader
from mcrgan.models import get_models
from mcrgan.default import _C as config
from mcrgan.default import update_config
from utils.utils import sort_dataset, compute_accuracy, extract_features2
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from tqdm import tqdm
import os

start_time = time.time()
#global_step = 0
epsilon_noise = 0.1
num_steps = 1000
n_dis = 1
n_dis2 = 1
numclasses =10
lr1 = 1e-3
lr2 = 1e-3
log_data = metric_log.MetricLog()
data_time = time.time()
iter_dataloader = iter(dataloader)
data, label = next(iter_dataloader)
data_time = time.time() - data_time
real_cpu = data.to(device)
real_label = label.clone().detach()

netD, netG, netV = get_BNN_models(DATASET, device, LABEL)
net_noise = get_noise(DATASET, device, LABEL)
fed_state_dict = get_netG(netD,netV,net_noise)
netG.load_state_dict(fed_state_dict)
optD = optim.Adam(netD.parameters(), lr1, betas=(0.5, 0.999))
optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))

Acc_results_D = []
Acc_results_G = []


transform = transforms.Compose(
            [transforms.Resize(32),
             transforms.ToTensor(),
             transforms.Normalize(0.5, 0.5)])

trainset = datasets.MNIST(root=ROOT, train=True, download=True, transform=transform)
testset = datasets.MNIST(root=ROOT, train=False, download=True, transform=transform)

testloader = torch.utils.data.DataLoader(
        testset, batch_size=DATA_SAMPLE, shuffle=False, num_workers=WORKERS)

trainloader = torch.utils.data.DataLoader(
        trainset, batch_size=DATA_SAMPLE, shuffle=False, num_workers=WORKERS)
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
n_comp = 10



In [6]:
# golden search algorithm to determined the radius of the sample ball, due to traditional updating netV is too slow
# no meaning for n_dis =2 for our case, just to have output for every update
# sample too much would arise cuda memory error
def gs_radius(radius1,radius2,netD,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=1,gam3=1):
    mcr_gan_loss=MCRGANloss(gam1=1., gam2=1., gam3=gam3, eps=0.5, numclasses=numclasses, mode=mode, rho=None)
    #optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))
    
    Z = netD(real_cpu)
    net_noise = get_noise(DATASET, device, LABEL)
    err1 = 0
    err2 = 1000
    while abs(err1 - err2)> tor or abs(radius1-radius2)>tor:
        a1 = radius2 - lam*(radius2-radius1)
        a2 = radius1 + lam*(radius2-radius1)
        fed_state_dict = get_netG(netD,netV,net_noise,a1)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err1 = errG1
        
        fed_state_dict = get_netG(netD,netV,net_noise,a2)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err2 = errG2
        
        if err1 > err2:
            radius1 = a1
            a1 = a2
            err1 = err2
            a2 = radius1 + lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a2)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err2 = errG2
            #print('r1',radius1)
            #print(errG2, errG_EC2)
        else:
            radius2 = a2
            a2 = a1
            a1 = radius2 - lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a1)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err1 = errG1
            #print('r2',radius2)
            #print(errG1, errG_EC1)
         
    return radius1,radius2

def gs_radius_noise(radius1,radius2,netD,noise, real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=1,gam3=1):
    mcr_gan_loss=MCRGANloss(gam1=1., gam2=1., gam3=gam3, eps=0.5, numclasses=numclasses, mode=mode, rho=None)
    #optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))
    
    Z = netD(real_cpu)
    err1 = 0
    err2 = 1000
    while abs(err1 - err2)> tor or abs(radius1-radius2)>tor:
        a1 = radius2 - lam*(radius2-radius1)
        a2 = radius1 + lam*(radius2-radius1)
        fed_state_dict = get_netG(netD,netV,net_noise,a1)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err1 = errG1
        
        fed_state_dict = get_netG(netD,netV,net_noise,a2)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err2 = errG2
        
        if err1 > err2:
            radius1 = a1
            a1 = a2
            err1 = err2
            a2 = radius1 + lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a2)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err2 = errG2
            #print('r1',radius1)
            #print(errG2, errG_EC2)
        else:
            radius2 = a2
            a2 = a1
            a1 = radius2 - lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a1)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err1 = errG1
            #print('r2',radius2)
            #print(errG1, errG_EC1)
         
    return radius1,radius2


def gs_radius2(radius1,radius2, netD,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=1,gam3=1):
    mcr_gan_loss=MCRGANloss(gam1=1., gam2=1., gam3=gam3, eps=0.5, numclasses=numclasses, mode=mode, rho=None)
    #optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))
    
    Z = netD(real_cpu)
    net_noise = get_noise(DATASET, device, LABEL)
    net_noise1 = get_noise(DATASET, device, LABEL)
    net_noise_cauchy = get_cauchy_noise(netD,net_noise,net_noise1)
    net_noise.load_state_dict(net_noise_cauchy)
    #net_noise = get_noise(DATASET, device, LABEL)
    err1 = 0
    err2 = 1000
    while abs(err1 - err2)> tor or abs(radius1-radius2)>tor:
        a1 = radius2 - lam*(radius2-radius1)
        a2 = radius1 + lam*(radius2-radius1)
        fed_state_dict = get_netG(netD,netV,net_noise,a1)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err1 = errG1
        
        fed_state_dict = get_netG(netD,netV,net_noise,a2)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err2 = errG2
        
        if err1 > err2:
            radius1 = a1
            a1 = a2
            err1 = err2
            a2 = radius1 + lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a2)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err2 = errG2
            #print('r1',radius1)
            #print(errG2, errG_EC2)
        else:
            radius2 = a2
            a2 = a1
            a1 = radius2 - lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a1)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err1 = errG1
            #print('r2',radius2)
            #print(errG1, errG_EC1)
         
    return radius1,radius2

def gs_radius3(radius1,radius2, netD,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=1,gam3=1):
    mcr_gan_loss=MCRGANloss(gam1=1., gam2=1., gam3=gam3, eps=0.5, numclasses=numclasses, mode=mode, rho=None)
    #optG = optim.Adam(netG.parameters(), lr2, betas=(0.5, 0.999))
    
    Z = netD(real_cpu)
    net_noise = get_noise_uniform(DATASET, device, LABEL)

    #net_noise = get_noise(DATASET, device, LABEL)
    err1 = 0
    err2 = 1000
    while abs(err1 - err2)> tor or abs(radius1-radius2)>tor:
        a1 = radius2 - lam*(radius2-radius1)
        a2 = radius1 + lam*(radius2-radius1)
        fed_state_dict = get_netG(netD,netV,net_noise,a1)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err1 = errG1
        
        fed_state_dict = get_netG(netD,netV,net_noise,a2)
        netG.load_state_dict(fed_state_dict)
        Z_bar = netG(real_cpu)
        errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
        err2 = errG2
        
        if err1 > err2:
            radius1 = a1
            a1 = a2
            err1 = err2
            a2 = radius1 + lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a2)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG2, errG_EC2 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err2 = errG2
            #print('r1',radius1)
            #print(errG2, errG_EC2)
        else:
            radius2 = a2
            a2 = a1
            a1 = radius2 - lam*(radius2-radius1)
            fed_state_dict = get_netG(netD,netV,net_noise,a1)
            netG.load_state_dict(fed_state_dict)
            Z_bar = netG(real_cpu)
            errG1, errG_EC1 = mcr_gan_loss(Z, Z_bar, real_label, n_dis2 - 1, n_dis2)
            err1 = errG1
            #print('r2',radius2)
            #print(errG1, errG_EC1)
         
    return radius1,radius2

In [8]:
# load_model
netD = torch.load('FMNIST_NetD_changing2-mode11.pth') 
netV = torch.load('FMNIST_NetV_changing2-mode11.pth') 
netG = torch.load('FMNIST_NetG_changing2-mode11.pth')

mode = 11
gam3 = 1
net_noise1 = get_noise(DATASET, device, LABEL)

In [9]:
#sample the length of radius,gaussian noise
mode = 11
gam3 = 1
L_noise= []
epsilon = 0.05
L_noise1= []
epsilon1 = 0.1

L_noise2= []
epsilon2 = 0.2

L_noise3= []
epsilon3 = 0.3
L_noise4= []
epsilon4 = 0.4
L_noise5= []
epsilon5 = 0.5
L_noise6= []
epsilon6 = 0.6
L_noise7= []
epsilon7 = 0.7
L_noise8= []
epsilon8 = 0.8
L_noise9= []
epsilon9 = 0.9

for _ in range(20):
    r1,r2 = gs_radius(0,100,netD,(1-epsilon)*real_cpu+epsilon*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
    print(r1)
    L_noise.append(r1)
print(max(L_noise),min(L_noise),np.mean(L_noise),np.var(L_noise))

for _ in range(20):
    r1,r2 = gs_radius(0,100,netD,(1-epsilon1)*real_cpu+epsilon1*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
    print(r1)
    L_noise1.append(r1)
print(max(L_noise1),min(L_noise1),np.mean(L_noise1),np.var(L_noise1))

for _ in range(20):
    r1,r2 = gs_radius(0,100,netD,(1-epsilon2)*real_cpu+epsilon2*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
    print(r1)
    L_noise2.append(r1)
print(max(L_noise2),min(L_noise2),np.mean(L_noise2),np.var(L_noise2))

for _ in range(20):
    r1,r2 = gs_radius(0,100,netD,(1-epsilon3)*real_cpu+epsilon3*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
    print(r1)
    L_noise3.append(r1)
print(max(L_noise3),min(L_noise3),np.mean(L_noise3),np.var(L_noise3))

for _ in range(20):
    r1,r2 = gs_radius(0,100,netD,(1-epsilon4)*real_cpu+epsilon4*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
    print(r1)
    L_noise4.append(r1)
print(max(L_noise4),min(L_noise4),np.mean(L_noise4),np.var(L_noise4))

for _ in range(20):
    r1,r2 = gs_radius(0,100,netD,(1-epsilon5)*real_cpu+epsilon5*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
    print(r1)
    L_noise5.append(r1)
print(max(L_noise5),min(L_noise5),np.mean(L_noise5),np.var(L_noise5))

for _ in range(20):
    r1,r2 = gs_radius(0,100,netD,(1-epsilon6)*real_cpu+epsilon6*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
    print(r1)
    L_noise6.append(r1)
print(max(L_noise6),min(L_noise6),np.mean(L_noise6),np.var(L_noise6))

for _ in range(20):
    r1,r2 = gs_radius(0,100,netD,(1-epsilon7)*real_cpu+epsilon7*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
    print(r1)
    L_noise7.append(r1)
print(max(L_noise7),min(L_noise7),np.mean(L_noise7),np.var(L_noise7))

for _ in range(20):
    r1,r2 = gs_radius(0,100,netD,(1-epsilon8)*real_cpu+epsilon8*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
    print(r1)
    L_noise8.append(r1)
print(max(L_noise8),min(L_noise8),np.mean(L_noise8),np.var(L_noise8))

for _ in range(20):
    r1,r2 = gs_radius(0,100,netD,(1-epsilon9)*real_cpu+epsilon9*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
    print(r1)
    L_noise9.append(r1)
print(max(L_noise9),min(L_noise9),np.mean(L_noise9),np.var(L_noise9))

0.5399485465557736
0.5424712933485455
0.537423408091128
0.574921275750062
0.5776721988376644
0.5887298249190436
0.5714346652256698
0.570471258187195
0.5496722339990149
0.5896924197237676
0.5006754235308543
0.5591691326750252
0.537423408091128
0.5368278308306048
0.5302226591895536
0.5720302424861929
0.56290136529888
0.5597646367020436
0.5827175286786548
0.5267361218986661
0.5896924197237676 0.5006754235308543 0.5555452737009734 0.0005320195903271521
0.5531598938351021
0.5415077678328297
0.5185653936036185
0.5006754235308543
0.5582057256365505
0.5109966232133467
0.5424708646597415
0.550041761492553
0.5374232163803822
0.5689106041388031
0.5352686095535487
0.5623058612718617
0.5547188781341001
0.567351811550551
0.5154471880275645
0.5683152185890258
0.5521947438137286
0.574921275750062
0.5562782911600511
0.5302226591895536
0.574921275750062 0.5006754235308543 0.5449490905681914 0.0004215791636386577
0.5012708090806316
0.45850500475171035
0.5377940582297573
0.5176019865651438
0.5352689197651

In [10]:
mode = 11
gam3 = 1

#gaussian
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon)*real_cpu+epsilon*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon1)*real_cpu+epsilon1*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon2)*real_cpu+epsilon2*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon3)*real_cpu+epsilon3*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon4)*real_cpu+epsilon4*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon5)*real_cpu+epsilon5*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon6)*real_cpu+epsilon6*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon7)*real_cpu+epsilon7*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon8)*real_cpu+epsilon8*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon9)*real_cpu+epsilon9*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)

0.5830881788172841
0.5811585443796569
0.574921275750062
0.5562782911600511
0.5739576770008413
0.660116628412574
0.8976033626765376
1.7894219768918669
2.4430088977448285
2.2784965215500357
2.1552385842916744


In [11]:
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon)*real_cpu+epsilon*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon1)*real_cpu+epsilon1*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon2)*real_cpu+epsilon2*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon3)*real_cpu+epsilon3*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon4)*real_cpu+epsilon4*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon5)*real_cpu+epsilon5*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon6)*real_cpu+epsilon6*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon7)*real_cpu+epsilon7*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon8)*real_cpu+epsilon8*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon9)*real_cpu+epsilon9*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)

5.089413453278834
4.992229288921812
4.6918882388573575
4.592769015647028
4.576794840998687
4.558169877407271
4.597806147434111
4.589046104007402
4.606350082277689
4.611622702946254


In [12]:
#fake_noise_uniform

r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon)*real_cpu+epsilon*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon1)*real_cpu+epsilon1*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon2)*real_cpu+epsilon2*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon3)*real_cpu+epsilon3*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon4)*real_cpu+epsilon4*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon5)*real_cpu+epsilon5*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon6)*real_cpu+epsilon6*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon7)*real_cpu+epsilon7*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon8)*real_cpu+epsilon8*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon9)*real_cpu+epsilon9*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)

0.5795993231026008
0.5714346652256698
0.5547188781341001
0.5735896555121439
0.6595211243855557
0.8777850726072888
1.4090745222860772
1.7063976301888517
1.6769798593674725
1.6229297135469178


In [13]:
#fake_noise_laplace

r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon)*real_cpu+epsilon*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon1)*real_cpu+epsilon1*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon2)*real_cpu+epsilon2*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon3)*real_cpu+epsilon3*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon4)*real_cpu+epsilon4*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon5)*real_cpu+epsilon5*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon6)*real_cpu+epsilon6*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon7)*real_cpu+epsilon7*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon8)*real_cpu+epsilon8*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon9)*real_cpu+epsilon9*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)

0.5846472815935231
0.5770760010541364
0.5648286080646335
0.5811585443796569
0.6564032290210646
0.8947125211615635
2.5996755858499676
4.137394350003416
4.0376596350602965
3.404447395345666


In [14]:
#fake_noise_gumbel
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon)*real_cpu+epsilon*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon1)*real_cpu+epsilon1*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon2)*real_cpu+epsilon2*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon3)*real_cpu+epsilon3*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon4)*real_cpu+epsilon4*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon5)*real_cpu+epsilon5*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon6)*real_cpu+epsilon6*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon7)*real_cpu+epsilon7*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon8)*real_cpu+epsilon8*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon9)*real_cpu+epsilon9*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)

0.5801950188639411
0.573589153551686
0.5550874015832554
0.574921275750062
0.6657583930151505
0.9038408962885323
1.7569802660775622
2.64243273534653
2.4760377964545732
2.309978576820681


In [15]:
mode

11

In [16]:
# set mode as 1, test again
mode = 1
gam3 = 1

#gaussian
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon)*real_cpu+epsilon*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon1)*real_cpu+epsilon1*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon2)*real_cpu+epsilon2*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon3)*real_cpu+epsilon3*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon4)*real_cpu+epsilon4*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon5)*real_cpu+epsilon5*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon6)*real_cpu+epsilon6*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon7)*real_cpu+epsilon7*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon8)*real_cpu+epsilon8*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon9)*real_cpu+epsilon9*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)

#cauchy
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon)*real_cpu+epsilon*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon1)*real_cpu+epsilon1*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon2)*real_cpu+epsilon2*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon3)*real_cpu+epsilon3*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon4)*real_cpu+epsilon4*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon5)*real_cpu+epsilon5*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon6)*real_cpu+epsilon6*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon7)*real_cpu+epsilon7*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon8)*real_cpu+epsilon8*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon9)*real_cpu+epsilon9*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)


#fake_noise_uniform

r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon)*real_cpu+epsilon*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon1)*real_cpu+epsilon1*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon2)*real_cpu+epsilon2*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon3)*real_cpu+epsilon3*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon4)*real_cpu+epsilon4*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon5)*real_cpu+epsilon5*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon6)*real_cpu+epsilon6*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon7)*real_cpu+epsilon7*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon8)*real_cpu+epsilon8*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon9)*real_cpu+epsilon9*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)

#fake_noise_laplace

r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon)*real_cpu+epsilon*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon1)*real_cpu+epsilon1*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon2)*real_cpu+epsilon2*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon3)*real_cpu+epsilon3*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon4)*real_cpu+epsilon4*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon5)*real_cpu+epsilon5*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon6)*real_cpu+epsilon6*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon7)*real_cpu+epsilon7*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon8)*real_cpu+epsilon8*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon9)*real_cpu+epsilon9*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)

#fake_noise_gumbel
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon)*real_cpu+epsilon*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon1)*real_cpu+epsilon1*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon2)*real_cpu+epsilon2*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon3)*real_cpu+epsilon3*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon4)*real_cpu+epsilon4*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon5)*real_cpu+epsilon5*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon6)*real_cpu+epsilon6*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon7)*real_cpu+epsilon7*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon8)*real_cpu+epsilon8*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon9)*real_cpu+epsilon9*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)

5.463517997663751
5.5710259175900445
6.021239298009232
8.868291154322423
13.603804915418005
25.974783639955128
99.86163004839798
99.9541307144992
99.92674126221517
99.9858271803377
99.97298440179843
99.82512261906379
99.515059318907
99.79712953798506
99.47987031864572
99.7735967993125
99.91449506495722
99.76230643252302
99.84277497361833
99.93550230966267
99.92362312987262
5.569835028013249
6.129121693825102
8.121788081696176
13.833377445616328
46.70840428065945
99.88242207034902
99.99183715817787
99.99243273543838
99.9720208030492
99.98211347073463
5.480820158509809
5.662926131744149
8.424443968848072
14.73582714744991
99.83557429795026
99.98330436031141
99.96949805625643
99.89838491613325
99.98211347073463
99.80506671332368
5.501231397227799
5.629896731473516
7.875452809796939
12.165281131582077
38.70409645427936
99.89155226195389
99.98018646494693
99.99339614247687
99.97261618859898
99.94618959585513


In [17]:
# set mode as 1, test again
mode = 1
gam3 = 0.2

#gaussian
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon)*real_cpu+epsilon*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon1)*real_cpu+epsilon1*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon2)*real_cpu+epsilon2*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon3)*real_cpu+epsilon3*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon4)*real_cpu+epsilon4*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon5)*real_cpu+epsilon5*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon6)*real_cpu+epsilon6*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon7)*real_cpu+epsilon7*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon8)*real_cpu+epsilon8*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon9)*real_cpu+epsilon9*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)

#cauchy
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon)*real_cpu+epsilon*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon1)*real_cpu+epsilon1*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon2)*real_cpu+epsilon2*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon3)*real_cpu+epsilon3*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon4)*real_cpu+epsilon4*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon5)*real_cpu+epsilon5*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon6)*real_cpu+epsilon6*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon7)*real_cpu+epsilon7*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon8)*real_cpu+epsilon8*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon9)*real_cpu+epsilon9*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)


#fake_noise_uniform

r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon)*real_cpu+epsilon*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon1)*real_cpu+epsilon1*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon2)*real_cpu+epsilon2*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon3)*real_cpu+epsilon3*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon4)*real_cpu+epsilon4*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon5)*real_cpu+epsilon5*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon6)*real_cpu+epsilon6*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon7)*real_cpu+epsilon7*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon8)*real_cpu+epsilon8*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon9)*real_cpu+epsilon9*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)

#fake_noise_laplace

r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon)*real_cpu+epsilon*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon1)*real_cpu+epsilon1*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon2)*real_cpu+epsilon2*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon3)*real_cpu+epsilon3*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon4)*real_cpu+epsilon4*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon5)*real_cpu+epsilon5*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon6)*real_cpu+epsilon6*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon7)*real_cpu+epsilon7*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon8)*real_cpu+epsilon8*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon9)*real_cpu+epsilon9*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)

#fake_noise_gumbel
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon)*real_cpu+epsilon*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon1)*real_cpu+epsilon1*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon2)*real_cpu+epsilon2*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon3)*real_cpu+epsilon3*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon4)*real_cpu+epsilon4*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon5)*real_cpu+epsilon5*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon6)*real_cpu+epsilon6*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon7)*real_cpu+epsilon7*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon8)*real_cpu+epsilon8*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon9)*real_cpu+epsilon9*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)

0.9170803545479528
0.9032455107387549
0.8548471438909666
0.6789691930431282
0.9475899925777715
4.028523638119587
5.797472929640106
6.4144816086121486
5.605627182519276
4.6170375603189315
4.110006905684685
8.313626943425843
8.634933010632416
8.591781663760278
8.506262342529224
8.536401211995901
8.547090624877837
8.555027800216449
8.513461777274738
8.443085136822111
8.488365991950241
0.8894364596093155
0.8202503242276409
0.7104424298221675
1.0253170043327788
4.373838133527508
5.559140046773541
15.682170154851518
99.45908823807837
99.91761145406993
99.97105739601072
0.9164855895211802
0.8834272207693924
0.7378353688655964
0.912373886003759
3.7850351467191787
5.1234151977419815
6.459760337212433
6.8108693134397384
6.2927909740622185
5.653427106478551
0.9007220702747794
0.8457079419984291
0.6651630074653732
0.9806273991679894
4.061560039766262
5.7352534882958395
6.6489344509399855
5.96801569918472
4.931208314523617
4.414071295308745


In [18]:
# set mode as 1, test again
mode = 1
gam3 = 0.1

#gaussian
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon)*real_cpu+epsilon*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon1)*real_cpu+epsilon1*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon2)*real_cpu+epsilon2*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon3)*real_cpu+epsilon3*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon4)*real_cpu+epsilon4*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon5)*real_cpu+epsilon5*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon6)*real_cpu+epsilon6*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon7)*real_cpu+epsilon7*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon8)*real_cpu+epsilon8*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon9)*real_cpu+epsilon9*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)

#cauchy
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon)*real_cpu+epsilon*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon1)*real_cpu+epsilon1*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon2)*real_cpu+epsilon2*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon3)*real_cpu+epsilon3*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon4)*real_cpu+epsilon4*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon5)*real_cpu+epsilon5*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon6)*real_cpu+epsilon6*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon7)*real_cpu+epsilon7*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon8)*real_cpu+epsilon8*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon9)*real_cpu+epsilon9*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)


#fake_noise_uniform

r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon)*real_cpu+epsilon*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon1)*real_cpu+epsilon1*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon2)*real_cpu+epsilon2*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon3)*real_cpu+epsilon3*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon4)*real_cpu+epsilon4*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon5)*real_cpu+epsilon5*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon6)*real_cpu+epsilon6*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon7)*real_cpu+epsilon7*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon8)*real_cpu+epsilon8*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon9)*real_cpu+epsilon9*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)

#fake_noise_laplace

r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon)*real_cpu+epsilon*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon1)*real_cpu+epsilon1*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon2)*real_cpu+epsilon2*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon3)*real_cpu+epsilon3*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon4)*real_cpu+epsilon4*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon5)*real_cpu+epsilon5*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon6)*real_cpu+epsilon6*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon7)*real_cpu+epsilon7*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon8)*real_cpu+epsilon8*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon9)*real_cpu+epsilon9*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)

#fake_noise_gumbel
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon)*real_cpu+epsilon*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon1)*real_cpu+epsilon1*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon2)*real_cpu+epsilon2*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon3)*real_cpu+epsilon3*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon4)*real_cpu+epsilon4*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon5)*real_cpu+epsilon5*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon6)*real_cpu+epsilon6*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon7)*real_cpu+epsilon7*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon8)*real_cpu+epsilon8*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon9)*real_cpu+epsilon9*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)

0.026052382835709986
0.028574892650423716
0.02041123871798922
0.04215595928108412
0.06316695544321596
0.10941214890563608
3.804495046110393
4.30366457163063
3.917164454462362
3.4660642573539837
3.184591668992402
6.780930391224707
6.999047795390325
7.0637865418590655
6.918143870480184
6.840791335037948
6.862168034112345
6.889785397708806
6.83790030177408
6.858304391811598
6.822156859198416
0.025456997285932606
0.017886912648722733
0.019447639968768575
0.04215595928108412
0.05536675930906181
2.9332187187249947
3.033534089958093
2.8043332142043185
2.628255662728393
2.519770183637946
0.033621271703222814
0.037110008917088974
0.07133142170928743
0.12321945715206291
0.21164013514438815
3.7673745941107346
4.935291359809594
5.633979274799036
5.338912232830861
4.774291052374993
0.020779260206686564
0.012245266546963311
0.0314665463991484
0.07541365476151511
0.16346248668387062
3.9043225614430668
4.759522345302325
4.3483504246922
3.809045135253425
3.535853592818099


In [19]:
# set mode as 1, test again
mode = 1
gam3 = 10

#gaussian
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon)*real_cpu+epsilon*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon1)*real_cpu+epsilon1*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon2)*real_cpu+epsilon2*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon3)*real_cpu+epsilon3*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon4)*real_cpu+epsilon4*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon5)*real_cpu+epsilon5*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon6)*real_cpu+epsilon6*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon7)*real_cpu+epsilon7*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon8)*real_cpu+epsilon8*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon9)*real_cpu+epsilon9*fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)

#cauchy
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon)*real_cpu+epsilon*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon1)*real_cpu+epsilon1*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon2)*real_cpu+epsilon2*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon3)*real_cpu+epsilon3*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon4)*real_cpu+epsilon4*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon5)*real_cpu+epsilon5*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon6)*real_cpu+epsilon6*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon7)*real_cpu+epsilon7*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon8)*real_cpu+epsilon8*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon9)*real_cpu+epsilon9*fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)


#fake_noise_uniform

r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon)*real_cpu+epsilon*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon1)*real_cpu+epsilon1*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon2)*real_cpu+epsilon2*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon3)*real_cpu+epsilon3*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon4)*real_cpu+epsilon4*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon5)*real_cpu+epsilon5*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon6)*real_cpu+epsilon6*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon7)*real_cpu+epsilon7*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon8)*real_cpu+epsilon8*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon9)*real_cpu+epsilon9*fake_noise_uniform,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)

#fake_noise_laplace

r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon)*real_cpu+epsilon*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon1)*real_cpu+epsilon1*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon2)*real_cpu+epsilon2*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon3)*real_cpu+epsilon3*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon4)*real_cpu+epsilon4*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon5)*real_cpu+epsilon5*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon6)*real_cpu+epsilon6*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon7)*real_cpu+epsilon7*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon8)*real_cpu+epsilon8*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon9)*real_cpu+epsilon9*fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)

#fake_noise_gumbel
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon)*real_cpu+epsilon*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon1)*real_cpu+epsilon1*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon2)*real_cpu+epsilon2*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon3)*real_cpu+epsilon3*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon4)*real_cpu+epsilon4*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon5)*real_cpu+epsilon5*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon6)*real_cpu+epsilon6*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon7)*real_cpu+epsilon7*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon8)*real_cpu+epsilon8*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)
r1,r2 = gs_radius_noise(0,100,netD,net_noise1,(1-epsilon9)*real_cpu+epsilon9*fake_noise_gumbel,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
print(r1)

12.08229495535862
12.406950807499907
13.11810052594683
21.24912066382192
64.16973811836526
99.988946124914
99.9541307144992
99.9541307144992
99.98582799257144
99.9858271803377
99.97298440179843
99.88049482758325
99.98835073936422
99.92518235114969
99.93839396341141
99.92637324072646
99.9541307144992
99.9547262185262
99.98115006369615
99.93550230966267
99.9541307144992
12.1429405967804
13.230475102049366
19.66065462547021
98.58275525127948
99.87485349175479
99.9914686347287
99.9933962609541
99.99243273543838
99.9720208030492
99.98115006369615
12.141810163927119
13.37704049610241
23.331333215550192
99.98426776734989
99.9541307144992
99.98619601406016
99.98211347073463
99.9827088562844
99.98211347073463
99.95821325782472
12.50821945016118
13.055895663247993
21.238416671186492
99.75533366816737
99.98990972366322
99.988946124914
99.98330467052298
99.98738690363693
99.97261618859898
99.98211347073463


In [20]:
#sample the length of radius,gaussian noise, cauchy noise, uniform noise.
mode = 11
gam3 = 1
L_0= []
L_1= []
L_2= []
for _ in range(20):
    r1,r2 = gs_radius(0,100,netD,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
    print(r1)
    L_0.append(r1)
    
for _ in range(20):
    r1,r2 = gs_radius2(0,100,netD,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
    print(r1)
    L_1.append(r1)
    
for _ in range(20):
    r1,r2 = gs_radius3(0,100,netD,real_cpu,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
    print(r1)
    L_2.append(r1)

0.5617104757220843
0.5271041433873636
0.5311860662280283
0.5817539299294342
0.5663875923636315
0.5393529692952505
0.49406936636981796
0.5358644237921301
0.5959299533357623
0.5541234925843228
0.5481128209730639
0.5349005880648513
0.49911632075461526
0.5257725231494456
0.5195291108300802
0.5377940582297573
0.552562765302426
0.6019460015940475
0.5481128209730639
0.5710666437369724
2.0496136318312153e-05
2.0052023568305115
4.401490084764663e-06
20.09370506976833
0.004449250672728261
19.715621614130487
10.565286871956529
23.735552523925264
0
0.1786046644556387
0
8.29397201427475
11.143734473684235
16.101580531733468
0
0
0
22.04223899011414
5.9800581848691495e-06
20.471859540849962


ValueError: 

In [21]:
L_0

[0.5617104757220843,
 0.5271041433873636,
 0.5311860662280283,
 0.5817539299294342,
 0.5663875923636315,
 0.5393529692952505,
 0.49406936636981796,
 0.5358644237921301,
 0.5959299533357623,
 0.5541234925843228,
 0.5481128209730639,
 0.5349005880648513,
 0.49911632075461526,
 0.5257725231494456,
 0.5195291108300802,
 0.5377940582297573,
 0.552562765302426,
 0.6019460015940475,
 0.5481128209730639,
 0.5710666437369724]

In [22]:
L_1

[2.0496136318312153e-05,
 2.0052023568305115,
 4.401490084764663e-06,
 20.09370506976833,
 0.004449250672728261,
 19.715621614130487,
 10.565286871956529,
 23.735552523925264,
 0,
 0.1786046644556387,
 0,
 8.29397201427475,
 11.143734473684235,
 16.101580531733468,
 0,
 0,
 0,
 22.04223899011414,
 5.9800581848691495e-06,
 20.471859540849962]

In [23]:
L_2

[]

In [24]:
#sample the length of radius
L_gaussian= []
for _ in range(20):
    r1,r2 = gs_radius(0,100,netD,fake_noise_normal,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
    print(r1)
    L_gaussian.append(r1)
    
print(max(L_gaussian),min(L_gaussian),np.mean(L_gaussian),np.var(L_gaussian))

2.8496148821510228
2.743268615060636
2.4480544193347145
2.2699639606616477
2.528921456927839
2.8441941462090106
2.954002661099339
2.258081029353192
2.5471839673883303
3.2197927663538963
2.451172551677263
2.604721536128657
2.5833356350558545
2.2102529495460828
2.8828702590550224
2.4358071448224523
2.468470148509615
2.1894710606894643
2.520733782387167
2.1935530567636348
3.2197927663538963 2.1894710606894643 2.5601733014587422 0.07578035169301947


In [25]:
#sample the length of radius
L_cauchy= []
for _ in range(20):
    r1,r2 = gs_radius(0,100,netD,fake_noise_cauchy,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
    print(r1)
    L_cauchy.append(r1)
    
print(max(L_cauchy),min(L_cauchy),np.mean(L_cauchy),np.var(L_cauchy))

4.507612960142069
5.851363937511906
4.137394350003416
4.842533115549504
4.815493928452844
4.78461132109956
4.802279687179763
4.793747391273776
4.41036477951542
4.429814044213204
4.1210139061439905
4.531513173051983
4.509172181419125
4.538351395589126
4.560692387221985
4.8419320431647135
4.628324101995077
5.592774152522848
4.8460145864902335
4.914867241231461
5.851363937511906 4.1210139061439905 4.7229935341886 0.16104679751416048


In [26]:
#sample the length of radius
L_laplace= []
for _ in range(20):
    r1,r2 = gs_radius(0,100,netD,fake_noise_laplace,real_label,tor=1e-3,n_dis=2,lam=0.618,mode=mode,gam3=gam3)
    print(r1)
    L_laplace.append(r1)
    
print(max(L_laplace),min(L_laplace),np.mean(L_laplace),np.var(L_laplace))

3.184161876486098
2.9816262144229877
3.3235780389291256
3.292474877042023
3.85076595835795
3.402878972070206
2.8523649930048744
3.2487457389081316
3.6557798040938327
3.4359213259339443
3.7601702122151615
3.9970529568041315
3.4745969367196126
3.353517198122213
3.544392652813406
4.128861287054684
3.545951563878899
3.50571584629619
3.443716958377682
3.758241463444568
4.128861287054684 2.8523649930048744 3.4870257437487857 0.09444914635821122



# other data sets, using same noise to get different r for diff data
import time
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
DATASET2 = 'FMNIST'
ROOT2    = 'pth/to/the/dataset'
BATCH_SIZE = 2048
WORKERS = 8
DATA_SAMPLE = 50000
LABEL = 0

dataloader2, dataset2 = get_dataloader(
        data_name=DATASET2,
        root=ROOT2,
        batch_size=BATCH_SIZE,
        num_workers=WORKERS)

data_time = time.time()
iter_dataloader2 = iter(dataloader2)
data2, label2 = next(iter_dataloader2)
data_time = time.time() - data_time
real_cpu2 = data2.to(device)
real_label2 = label2.clone().detach()


device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
DATASET3 = 'cifar10'
ROOT3    = 'pth/to/the/dataset'
BATCH_SIZE = 2048
WORKERS = 8
DATA_SAMPLE = 50000
LABEL = 0

dataloader3, dataset3 = get_dataloader(
        data_name=DATASET3,
        root=ROOT3,
        batch_size=BATCH_SIZE,
        num_workers=WORKERS)

data_time = time.time()
iter_dataloader3 = iter(dataloader3)
data3, label3 = next(iter_dataloader3)
data_time = time.time() - data_time
real_cpu3 = data3.to(device)
real_label3 = label3.clone().detach()

sample1 = real_cpu2[:,0,:,:]
sample2 = real_cpu2[:,1,:,:]
sample3 = real_cpu2[:,2,:,:]